# Lightweight Fine-Tuning Project

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

* PEFT technique:LoRA
* Model:bert-base-uncased
* Evaluation approach:epoch
* Fine-tuning dataset: imdb

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset


model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


dataset = load_dataset("imdb")
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(2000))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(1000))


def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)


tokenized_dataset = dataset.map(tokenize_function, batched=True)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

base_training_args = TrainingArguments(
        output_dir="/tmp/BertFineTuningClassifier/base_results",
        learning_rate=3e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

base_trainer = Trainer(
    model = model,
    args = base_training_args,
    train_dataset= tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer = tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

base_results = base_trainer.evaluate()
print("Base Accuracy:", base_results["eval_accuracy"])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Base Accuracy: 0.471


In [ ]:
for name, module in model.named_modules():
    print(name)



In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha = 16,
    target_modules = ["query", "value"],
    lora_dropout = 0.1,
    bias = "none",
    task_type = "SEQ_CLS"
)

lora_model = get_peft_model(model,lora_config)
lora_model.print_trainable_parameters()

trainable params: 297,988 || all params: 109,780,228 || trainable%: 0.27144050019644705


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
        output_dir="/tmp/BertFineTuningClassifier/tuning_results",
        learning_rate=3e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

trainer = Trainer(
    model = lora_model,
    args = training_args,
    train_dataset= tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer = tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.700700,0.652537,0.660000
2,0.520500,0.527727,0.759000
3,0.458000,0.479475,0.787000


TrainOutput(global_step=1500, training_loss=0.5858481879234314, metrics={'train_runtime': 597.9129, 'train_samples_per_second': 10.035, 'train_steps_per_second': 2.509, 'total_flos': 1584130498560000.0, 'train_loss': 0.5858481879234314, 'epoch': 3.0})

In [ ]:
# Saving the model
lora_model.save_pretrained("/tmp/LightTunedLoraModel_Final")

In [ ]:
import os

# Path where you saved the model
save_path = "/tmp/LightTunedLoraModel_Final"  # or your workspace path

# List all files and folders in the directory
files = os.listdir(save_path)
print(files)


['adapter_model.bin', 'adapter_config.json', 'README.md']


## Performing Inference with a PEFT Model

In [ ]:

finetuned_results = trainer.evaluate()
print("Fine-tuned Accuracy:", finetuned_results["eval_accuracy"])


print("Base Accuracy:", base_results["eval_accuracy"])


improvement = finetuned_results["eval_accuracy"] - base_results["eval_accuracy"]
print(f"Accuracy improvement: {improvement:.3f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuned Accuracy: 0.835
Base Accuracy: 0.52
Accuracy improvement: 0.315


Fine-tuned Accuracy: 0.787
Base Accuracy: 0.471
Accuracy improvement: 0.316
